In [114]:
import pandas as pd

cs = pd.read_csv('customer_satisfaction.csv')
ct = pd.read_csv('customer_transactions.csv')
mp = pd.read_csv('marketing_performance.csv')

문제 1: 채널별 고객 획득 비용(CAC) 및 생애 가치(LTV) 분석
목표: 마케팅 채널의 진정한 ROI를 평가하기 위해 CAC와 LTV를 계산하고 최적 투자 전략을 제안하세요.
1. 데이터 로드 및 전처리
2. 채널별 CAC 계산
    - 2024년 데이터만 사용 - (2024년 마케팅데이터만 사용 -> 24년 채녈별 마케팅비용/24년 신규 획득 고객 수)
    - 각 채널별 총 마케팅 비용 집계
    - 각 채널별 신규 고객 수 집계 (2024년 첫 거래 기준)
    - CAC = 총 마케팅 비용 / 신규 고객 수
    - organic과 referral 채널은 CAC = 0으로 처리
3. 고객별 LTV 계산 - 전체 구매 이력 기반
4. 채널별 LTV 및 ROI 분석 - (`24년 신규 고객들의 평균 LTV / 24년 CAC`)    
    - 채널별 평균 LTV 계산
    - ROI = LTV / CAC 계산 (organic, referral은 무한대 처리)
    - payback period = CAC / (월평균 구매금액) 계산
5. 시각화 및 전략 제안 (15분)
    - CAC vs LTV 산점도 차트
    - 채널별 ROI 막대 차트
    - 마케팅 예산 재배분 제안 (현재 vs 최적 배분)

In [115]:
# 24년 데이터만 뽑기
mp['month'] = pd.to_datetime(mp['month'])
mp24 = mp[mp['month'].dt.year == 2024]
# 24년 채널별 총마케팅 비용
marketing_sum = mp24.groupby('channel')['marketing_spend'].sum().reset_index()
marketing_sum
# 24년 채널별 신규 고객 수 집계
new_customer_sum = mp24.groupby('channel')['new_customers'].sum().reset_index()
new_customer_sum
# CAC- 1인당 마케팅 비용
cac = marketing_sum['marketing_spend'] / new_customer_sum['new_customers']
cac.index = marketing_sum['channel']
cac
# organic과 referral 채널은 CAC = 0으로 처리
cac.loc[['organic', 'referral']] = 0
cac = cac.reset_index()
cac.columns = ['channel', 'CAC']

# mp24에 CAC col 추가하기 위해서, 병합
mp24 = mp24.merge(cac[['channel', 'CAC']], on='channel', how='left')
mp24.head()

,month,channel,marketing_spend,new_customers,website_visits,conversions,CAC
0,2024-01-01,organic,0.000000,18,471,18,0.000000
1,2024-01-01,paid_search,8760.203892,18,991,18,436.826024
2,2024-01-01,social_media,5272.708407,15,992,15,327.630092
3,2024-01-01,email,1957.893749,12,1050,12,112.425752
4,2024-01-01,referral,1288.390831,6,983,6,0.000000


In [91]:
# 채널별 LTV 및 ROI 분석 - (`24년 신규 고객들의 평균 LTV / 24년 CAC`)    
#     - 채널별 평균 LTV 계산
#     - ROI = LTV / CAC 계산 (organic, referral은 무한대 처리)
#     - payback period = CAC / (월평균 구매금액) 계산

In [ ]:
# 24년도에 가입한 고객들의 거래만 뽑기
ct['transaction_date'] = pd.to_datetime(ct['registration_date'])
ct24 = ct[ct['transaction_date'].dt.year == 2024]

# 과거 매출 합해서 고객 ltv 구하기
customer_ltv = ct24.groupby('customer_id')['order_value'].sum().reset_index()
customer_ltv.columns = ['customer_id', 'ltv']

# ltv를 기존 df에 병합
ct24 = ct24.merge(customer_ltv, on='customer_id', how='left')
ct24

,customer_id,registration_date,acquisition_channel,customer_segment,transaction_date,order_value,product_category,ltv
0,3,2024-03-30,social_media,basic,2024-03-30,23.95,Fashion,72.52
1,3,2024-03-30,social_media,basic,2024-03-30,48.57,Fashion,72.52
2,4,2024-04-21,referral,standard,2024-04-21,82.92,Electronics,345.55
3,4,2024-04-21,referral,standard,2024-04-21,70.54,Electronics,345.55
4,4,2024-04-21,referral,standard,2024-04-21,80.38,Books,345.55
...,...,...,...,...,...,...,...,...
1144,998,2024-06-07,paid_search,standard,2024-06-07,109.13,Fashion,190.67
1145,998,2024-06-07,paid_search,standard,2024-06-07,81.54,Electronics,190.67
1146,999,2024-07-06,paid_search,basic,2024-07-06,25.21,Books,25.21
1147,1000,2024-05-04,social_media,standard,2024-05-04,67.35,Home,115.52


In [ ]:
# 24년도 채널별 평균 ltv 구하기
channel_ltv = ct24.groupby('acquisition_channel')['ltv'].mean().reset_index()
channel_ltv.rename(columns={'acquisition_channel': 'channel'}, inplace=True)

# 채널별 LTV 및 ROI 분석
# 채널별 마케팅 비용
channel_cac = mp24.groupby('channel')['CAC'].sum().reset_index()

# 채널별 ROI = LTV / CAC 계산 (organic, referral은 무한대 처리)
df = pd.merge(ct24, mp24, on='channel') ?????????????????????????????????????????
df['roi'] = df['ltv'] / df['CAC']
df

# payback period = CAC / (월평균 구매금액) 계산



KeyError: 'channel'

In [90]:
customer_ltv.head()

,customer_id,ltv
0,1,938.00
1,2,660.75
2,3,548.71
3,4,102.23
4,5,378.85


In [88]:
ct.head()

,customer_id,registration_date,acquisition_channel,customer_segment,transaction_date,order_value,product_category,ltv
0,1,2023-04-13,email,premium,2023-08-22,104.24,Books,938.0
1,1,2023-04-13,email,premium,2024-01-15,141.21,Electronics,938.0
2,1,2023-04-13,email,premium,2024-02-21,54.80,Electronics,938.0
3,1,2023-04-13,email,premium,2023-06-11,159.59,Home,938.0
4,1,2023-04-13,email,premium,2023-09-30,111.29,Fashion,938.0


In [87]:
customer_ltv

,customer_id,ltv
0,1,938.00
1,2,660.75
2,3,548.71
3,4,102.23
4,5,378.85
...,...,...
995,996,44.13
996,997,88.83
997,998,217.17
998,999,208.73


In [7]:
mp.head()

,month,channel,marketing_spend,new_customers,website_visits,conversions
0,2023-01-01,organic,0.000000,28,511,28
1,2023-01-01,paid_search,11986.882840,19,995,19
2,2023-01-01,social_media,4235.453095,17,1048,17
3,2023-01-01,email,1693.773605,17,1037,17
4,2023-01-01,referral,887.489645,10,958,10


In [86]:
ct.sort_values(['customer_id', 'transaction_date'])['order_value'].sum()


np.float64(397914.94999999995)

문제 2: 고객 코호트 분석 및 유지율 개선 전략
목표: "월별 가입 코호트"의 유지율과 매출 기여도를 분석하여 고객 유지 전략을 수립하세요.
1. 코호트 그룹 생성
2. 월별 유지율 계산
    - 각 거래의 코호트 대비 경과 개월 수 계산
    - 코호트별 Month 0, 1, 2, ... 12에서의 활성 고객 수
    - 유지율 테이블 생성 (코호트 × 경과월)
    - 평균 유지율 곡선 계산
3. 코호트별 매출 기여도 분석
    - 각 코호트의 월별 누적 매출 계산
    - 코호트별 고객 1인당 누적 매출 (ARPU) 계산
    - 6개월 후, 12개월 후 예상 LTV 추정
4. 유지율 히트맵 시각화
5. 개선 전략 수립
    - 유지율이 급격히 감소하는 시점 식별
    - 고성과 코호트 vs 저성과 코호트 특성 비교
    - 유지율 개선을 위한 액션 아이템 3가지 제안